# Introduction

In this workshop, we will go through the steps of training and deploying a **network traffic classification model**.  We will show how to train two version of models where we will deploy them to a production and shadow variant.  We will use SageMaker Shadow Tests to help manage the test between the production and shadow variants.  

## Contents

1) [Setup](#setup)
2) [Basic Training](#basic_training)
3) [Register the Models](#register)
4) [Create Endpoint Config](#create_endpoint)
5) [Deploy and Predict](#deploy)
6) [Create a Shadow Test](#shadow)
7) [Evaluate](#eval)

For training our model we will be using datasets <a href="https://registry.opendata.aws/cse-cic-ids2018/">CSE-CIC-IDS2018</a> by CIC and ISCX which are used for security testing and malware prevention.
These datasets include a huge amount of raw network traffic logs, plus pre-processed data where network connections have been reconstructed and  relevant features have been extracted using CICFlowMeter, a tool that outputs network connection features as CSV files. Each record is classified as benign traffic, or it can be malicious traffic, with a total number of 15 classes.

Starting from this featurized dataset, we have executed additional pre-processing for the purpose of this lab:
<ul>
    <li>Encoded class labels</li>
    <li>Replaced invalid string attribute values generated by CICFlowMeter (e.g. inf and Infinity)</li>
    <li>Executed one hot encoding of discrete attributes</li>
    <li>Remove invalid headers logged multiple times in the same CSV file</li>
    <li>Reduced the size of the featurized dataset to ~1.3GB (from ~6.3GB) to speed-up training, while making sure that all classes are well represented</li>
    <li>Executed stratified random split of the dataset into training (80%) and validation (20%) sets</li>
</ul>

Class are represented and have been encoded as follows (train + validation):


| Label                    | Encoded | N. records |
|:-------------------------|:-------:|-----------:|
| Benign                   |    0    |    1000000 |
| Bot                      |    1    |     200000 |
| DoS attacks-GoldenEye    |    2    |      40000 |
| DoS attacks-Slowloris    |    3    |      10000 |
| DDoS attacks-LOIC-HTTP   |    4    |     300000 |
| Infilteration            |    5    |     150000 |
| DDOS attack-LOIC-UDP     |    6    |       1730 |
| DDOS attack-HOIC         |    7    |     300000 |
| Brute Force -Web         |    8    |        611 |
| Brute Force -XSS         |    9    |        230 |
| SQL Injection            |   10    |         87 |
| DoS attacks-SlowHTTPTest |   11    |     100000 |
| DoS attacks-Hulk         |   12    |     250000 |
| FTP-BruteForce           |   13    |     150000 |
| SSH-Bruteforce           |   14    |     150000 |       

The final pre-processed dataset has been saved to a public Amazon S3 bucket for your convenience, and will represent the inputs to the training processes.
<a id='setup'></a>
### Let's get started!

First, we set some variables, including the AWS region we are working in, the IAM (Identity and Access Management) execution role of the notebook instance and the Amazon S3 bucket where we will store data, models, outputs, etc. We will use the Amazon SageMaker default bucket for the selected AWS region, and then define a key prefix to make sure all objects have share the same prefix for easier discoverability.

In [94]:
import os
import boto3
import sagemaker
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from IPython.display import display, clear_output
from sagemaker.sklearn.estimator import SKLearn
from sklearn.model_selection import train_test_split
from sagemaker.model_monitor import DataCaptureConfig
import pandas as pd
import numpy as np
import time
import glob

pd.options.display.max_columns = 100

region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker.Session().default_bucket()
prefix = 'xgboost-webtraffic'
os.environ["AWS_REGION"] = region

print(f'REGION:  {region}')
print(f'ROLE:    {role}')
print(f'BUCKET:  {bucket_name}')

REGION:  us-east-1
ROLE:    arn:aws:iam::431615879134:role/sagemaker-test-role
BUCKET:  sagemaker-us-east-1-431615879134


Now we can copy the dataset from the public Amazon S3 bucket to the Amazon SageMaker default bucket used in this workshop. To do this, we will leverage on the AWS Python SDK (boto3) as follows:

In [66]:
s3 = boto3.resource('s3')

source_bucket_name = "endtoendmlapp"
source_bucket_prefix = "aim362/data/"
source_bucket = s3.Bucket(source_bucket_name)

for s3_object in source_bucket.objects.filter(Prefix=source_bucket_prefix):
    copy_source = {
        'Bucket': source_bucket_name,
        'Key': s3_object.key
    }
    print('Copying {0} ...'.format(s3_object.key))
    s3.Bucket(bucket_name).copy(copy_source, prefix+'/data/'+s3_object.key.split('/')[-2]+'/'+s3_object.key.split('/')[-1])
    
print(f'Data copy from source bucket, {source_bucket_name}/{source_bucket_prefix}, to destination bucket {bucket_name}/{prefix}/data/, complete!')

Copying aim362/data/train/0.part ...
Copying aim362/data/train/1.part ...
Copying aim362/data/train/2.part ...
Copying aim362/data/train/3.part ...
Copying aim362/data/train/4.part ...
Copying aim362/data/train/5.part ...
Copying aim362/data/train/6.part ...
Copying aim362/data/train/7.part ...
Copying aim362/data/train/8.part ...
Copying aim362/data/train/9.part ...
Copying aim362/data/val/0.part ...
Copying aim362/data/val/1.part ...
Copying aim362/data/val/2.part ...
Copying aim362/data/val/3.part ...
Copying aim362/data/val/4.part ...
Copying aim362/data/val/5.part ...
Copying aim362/data/val/6.part ...
Copying aim362/data/val/7.part ...
Copying aim362/data/val/8.part ...
Copying aim362/data/val/9.part ...
Data copy from source bucket, endtoendmlapp/aim362/data/, to destination bucket sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/, complete!


Let's download some of the data to the notebook to quickly explore the dataset structure:

# Data

In [67]:
train_file_path = 's3://' + bucket_name + '/' + prefix + '/data/train/0.part'
val_file_path = 's3://' + bucket_name + '/' + prefix + '/data/val/'

print(train_file_path)
print(val_file_path)

s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/train/0.part
s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/


In [68]:
!mkdir -p data/train/ data/val/
!aws s3 cp {train_file_path} data/train/ 
!aws s3 cp {val_file_path} data/val/ --recursive

download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/train/0.part to data/train/0.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/1.part to data/val/1.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/0.part to data/val/0.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/2.part to data/val/2.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/3.part to data/val/3.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/5.part to data/val/5.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/4.part to data/val/4.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/7.part to data/val/7.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/6.part to data/val/6.part
download: s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/8.

In [69]:
df = pd.read_csv('data/train/0.part')
df

,Target,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,day,month,year,dayofweek,prot_0,prot_6,prot_17
0,0,445,64443,5,4,373,172,140,0,74.600000,70.283711,133,0,43.000000,62.753486,8457.086107,139.658303,8.055375e+03,1.105582e+04,21474,3,64403,1.610075e+04,1.073215e+04,21537,3,64398,2.146600e+04,129.201393,21547,21317,0,0,0,0,112,92,77.587946,62.070357,0,140,54.500000,64.198044,4121.388889,0,0,0,1,0,0,0,0,0.0,60.555556,74.600000,43.000000,0.0,0.0,0.0,0.0,0.0,0.0,5,373,4,172,8192,0,3,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0
1,12,80,1527,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,1309.757695,1.527000e+03,0.000000e+00,1527,1527,1527,1.527000e+03,0.000000e+00,1527,1527,0,0.000000e+00,0.000000,0,0,0,0,0,0,64,0,1309.757695,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,225,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,16,2,2018,4,0,1,0
2,7,80,5573,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,358.873138,5.573000e+03,0.000000e+00,5573,5573,5573,5.573000e+03,0.000000e+00,5573,5573,0,0.000000e+00,0.000000,0,0,0,0,0,0,40,0,358.873138,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,32738,-1,0,20,0.0,0.0,0,0,0.0,0.0,0,0,21,2,2018,2,0,1,0
3,12,80,44934,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,44.509725,4.493400e+04,0.000000e+00,44934,44934,44934,4.493400e+04,0.000000e+00,44934,44934,0,0.000000e+00,0.000000,0,0,0,0,0,0,64,0,44.509725,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,225,-1,0,32,0.0,0.0,0,0,0.0,0.0,0,0,16,2,2018,4,0,1,0
4,0,443,60108569,4,2,148,252,74,0,37.000000,42.723920,126,126,126.000000,0.000000,6.654625,0.099819,1.202171e+07,2.677679e+07,59921494,44882,60108569,2.003619e+07,3.454169e+07,59921494,93516,60013670,6.001367e+07,0.000000,60013670,60013670,1,0,0,0,80,40,0.066546,0.033273,0,126,67.714286,51.774235,2680.571429,0,1,0,0,1,0,0,0,0.0,79.000000,37.000000,126.000000,0.0,0.0,0.0,0.0,0.0,0.0,4,148,2,252,257,7010,1,20,93559.0,0.0,93559,93559,59921494.0,0.0,59921494,59921494,20,2,2018,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212208,14,22,298760,21,21,1912,2665,640,0,91.047619,139.679088,976,0,126.904762,263.574639,15319.989289,140.581068,7.286829e+03,2.230386e+04,122248,2,298722,1.493610e+04,3.049364e+04,122248,320,298752,1.493760e+04,33997.036726,126346,7,0,0,0,0,680,680,70.290534,70.290534,0,976,106.441860,207.291869,42969.919158,0,0,0,1,0,0,0,0,1.0,108.976190,91.047619,126.904762,0.0,0.0,0.0,0.0,0.0,0.0,21,1912,21,2665,26883,230,16,32,0.0,0.0,0,0,0.0,0.0,0,0,14,2,2018,2

In [70]:
%%time
val_csv_files = glob.glob("./data/val/*.part")
df_list = (pd.read_csv(file) for file in val_csv_files)
val_df= pd.concat(df_list, ignore_index=True)
val_df

CPU times: user 3.82 s, sys: 302 ms, total: 4.12 s
Wall time: 4.47 s


,Target,Dst Port,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,day,month,year,dayofweek,prot_0,prot_6,prot_17
0,4,80,41741569,2,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,4.791387e-02,4.170000e+07,0.000000,41700000,41700000,41700000,4.170000e+07,0.000000,41700000,41700000,0,0.000000,0.000000,0,0,0,0,0,0,40,0,0.047914,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,0,2049,-1,0,20,0.0,0.0,0,0,41700000.0,0.0,41700000,41700000,20,2,2018,1,0,1,0
1,4,80,1267608,3,4,20,964,20,0,6.666667,11.547005,964,0,241.000000,482.000000,776.265218,5.522212e+00,2.112680e+05,517266.616200,1267134,1,447,2.235000e+02,314.662518,446,1,1267602,422534.000000,731445.086800,1267134,22,0,0,0,0,72,92,2.366662,3.155550,0,964,123.000000,339.887376,115523.428600,0,0,1,1,0,0,0,1,1.0,140.571429,6.666667,241.000000,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0
2,13,21,1,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,2.000000e+06,1.000000e+00,0.000000,1,1,0,0.000000e+00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,40,20,1000000.000000,1000000.000000,0,0,0.000000,0.000000,0.000000,0,0,0,1,0,0,0,0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,26883,0,0,40,0.0,0.0,0,0,0.0,0.0,0,0,14,2,2018,2,0,1,0
3,13,21,1,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,2.000000e+06,1.000000e+00,0.000000,1,1,0,0.000000e+00,0.000000,0,0,0,0.000000,0.000000,0,0,0,0,0,0,40,20,1000000.000000,1000000.000000,0,0,0.000000,0.000000,0.000000,0,0,0,1,0,0,0,0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0,26883,0,0,40,0.0,0.0,0,0,0.0,0.0,0,0,14,2,2018,2,0,1,0
4,0,3389,1719730,8,7,1128,1581,661,0,141.000000,222.623321,1173,0,225.857143,430.098604,1575.247277,8.722299e+00,1.228379e+05,245415.138688,953191,4,1719730,2.456757e+05,390340.649099,1125099,4,1601462,266910.333333,336841.974375,953191,113295,0,0,0,0,172,152,4.651893,4.070406,0,1173,169.312500,317.731274,100953.162500,0,0,1,1,0,0,0,1,0.0,180.600000,141.000000,225.857143,0.0,0.0,0.0,0.0,0.0,0.0,8,1128,7,1581,8192,62872,5,20,0.0,0.0,0,0,0.0,0.0,0,0,28,2,2018,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583581,0,3389,2674228,8,7,1144,1581,677,0,143.000000,227.969923,1173,0,225.857143,430.098604,1018.985666,5.609095e+00,1.910163e+05,219001.515430,822108,3,2674228,3.820326e+05,409243.423924,1275528,3,2428015,404669.166667,217330.071145,822108,247487,0,0,0,0,172,152,2.991518,2.617578,0,1173,170.312500,319.402717,102018.095833,0,0,0,1,0,0,0,0,0.0,181.666667,143.000000,225.857143,0.0,0.0,0.0,0.0,0.0,0.0,8,1144,7,1581,64240,62856,5,20,0.0,0.0,0,0,0.0,0.0,0,0,15,2,2018,3,0,

In [71]:
newval_df, holdout = train_test_split(val_df, test_size=.2, random_state=42, stratify=val_df['Target'])
print(holdout.shape)
print(newval_df.shape)

(116718, 85)
(466868, 85)


In [72]:
holdout.to_csv('./data/holdout.csv',index=False)
newval_df.to_csv('./data/val/newval.csv',index=False)
del val_df, newval_df

In [73]:
val_data_path = f"s3://{bucket_name}/{prefix}/data/val/newval.csv"
!aws s3 cp ./data/val/newval.csv {val_data_path}

upload: data/val/newval.csv to s3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/data/val/newval.csv


<a id='basic_training'></a>
# Training

We will execute the training using the built in XGBoost algorithm.  Not that you can also use script mode if you need to have greater customization of the training process.  


In [74]:
container = sagemaker.image_uris.retrieve('xgboost',region,version='1.0-1')
print(container)

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3


In [75]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/data/train'.format(bucket_name, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/data/val/newval.csv'.format(bucket_name, prefix), content_type='csv')

## Model 1 - XGBoost

In [76]:
hyperparameters = {
    "max_depth": "3",
    "eta": "0.1",
    "gamma": "6",
    "min_child_weight": "6",
    "objective": "multi:softmax",
    "num_class": "15",
    "num_round": "10"
}

output_path = f's3://{bucket_name}/{prefix}/output/'

# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=role,
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path)



In [77]:
estimator.fit({'train': s3_input_train, 'validation': s3_input_validation})

2023-01-25 22:46:26 Starting - Starting the training job...
2023-01-25 22:46:50 Starting - Preparing the instances for trainingProfilerReport-1674686786: InProgress
......
2023-01-25 22:47:50 Downloading - Downloading input data......
2023-01-25 22:48:50 Training - Training image download completed. Training in progress.[2023-01-25 22:48:38.592 ip-10-0-249-251.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','


## Model 2 - Sklearn Random Forest

In [78]:
output_path = f's3://{bucket_name}/{prefix}/output/'

FRAMEWORK_VERSION = "0.23-1"

estimator2 = SKLearn(
    entry_point="randomforest.py",
    source_dir='./code/',
    role=role,
    instance_count=1,
    instance_type="ml.m5.2xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[
        {"Name": "Accuracy", "Regex": "Accuracy is: ([0-9.]+).*$"},
        {"Name": "WeightedF1", "Regex": "Weighted F1 Score is: ([0-9.]+).*$"}
    ],
    output_path=output_path,
    hyperparameters={
        "n-estimators": 50,
        "min-samples-leaf": 2
    }
)

In [79]:
estimator2.fit({'train': s3_input_train, 'validation': s3_input_validation})

2023-01-25 22:52:23 Starting - Starting the training job...
2023-01-25 22:52:47 Starting - Preparing the instances for trainingProfilerReport-1674687143: InProgress
......
2023-01-25 22:53:47 Downloading - Downloading input data.....2023-01-25 22:54:29,530 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-25 22:54:29,534 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 22:54:29,575 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-25 22:54:29,754 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 22:54:29,766 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 22:54:29,778 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 22:54:29,788 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_param

In order to make sure that our code works for inference, we can deploy the trained model and execute some inferences.

<a id='register'></a>
## Register our models

In [81]:
sm_client = boto3.Session().client('sagemaker')

In [123]:
estimator2.training_image_uri()

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

In [160]:
t_ = sm_client.describe_training_job(TrainingJobName='rf-scikit-2023-01-25-22-52-23-013')

In [153]:
estimator2.

In [146]:
estimator2.hyperparameters()['sagemaker_program']

'"randomforest.py"'

In [158]:
model_name1 = "PROD-XGBoost-Webtraffic"
model_name2 = "SHADOW-RandomForest-Webtraffic"

print(f"Prod model name: {model_name1}")
print(f"Shadow model name: {model_name2}")

resp = sm_client.create_model(
    ModelName=model_name1,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": container, 
                      "ModelDataUrl": estimator.model_data
                     }
)

resp = sm_client.create_model(
    ModelName=model_name2,
    ExecutionRoleArn=role,
    PrimaryContainer={
                      "Image": estimator2.training_image_uri(),
                      "ModelDataUrl": estimator2.model_data,
                      "Environment": {
                          "SAGEMAKER_SUBMIT_DIRECTORY":'/opt/ml/code',
                          "SAGEMAKER_PROGRAM":'"randomforest.py"',
                      },
                     }
)

Prod model name: PROD-XGBoost-Webtraffic
Shadow model name: SHADOW-RandomForest-Webtraffic


In [164]:
t_["HyperParameters"]["sagemaker_submit_directory"]

'"s3://sagemaker-us-east-1-431615879134/rf-scikit-2023-01-25-22-52-23-013/source/sourcedir.tar.gz"'

In [167]:
manual_response = sm_client.create_model(
    ModelName='test-skdeploy',
    PrimaryContainer={
        "Image": '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3',
        "Mode": "SingleModel",
        "ModelDataUrl": 's3://sagemaker-us-east-1-431615879134/xgboost-webtraffic/output/rf-scikit-2023-01-25-22-52-23-013/output/model.tar.gz',
        "Environment": {
            "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
            "SAGEMAKER_PROGRAM": t_["HyperParameters"]["sagemaker_program"],
            "SAGEMAKER_SUBMIT_DIRECTORY": 's3://sagemaker-us-east-1-431615879134/rf-scikit-2023-01-25-22-52-23-013/source/sourcedir.tar.gz',
        },
    },
    ExecutionRoleArn=role,
)

manual_response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:431615879134:model/test-skdeploy',
 'ResponseMetadata': {'RequestId': 'a7c34600-31ad-4e5f-bf19-087ed9776b9a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a7c34600-31ad-4e5f-bf19-087ed9776b9a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '75',
   'date': 'Thu, 26 Jan 2023 21:55:15 GMT'},
  'RetryAttempts': 0}}

In [169]:
endpoint_config_1_response = sm_client.create_endpoint_config(
    EndpointConfigName='sklearnendpoint',
    ProductionVariants=[
        {
            "VariantName": "AllTrafficVariant",
            "ModelName": 'test-skdeploy',
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.xlarge",
            "InitialVariantWeight": 1,
        },
    ],
)

In [171]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName='sklearnendpointdeploy',
    EndpointConfigName='sklearnendpoint',
)

<a id='create_endpoint'></a>
## Create Endpoint Config

Here we will create the endpoint configuration for the production endpoint.  We also include data capture of 100% of the input and output traffic to the production endpoint.  Note that we could also use the highlevel sagemaker SDK (estimator.deploy())

In [20]:
ep_config_name = "shadow-xgboost-epconfig"

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=ep_config_name,
    ProductionVariants=[
        {
            "VariantName": model_name1,
            "ModelName": model_name1,
            "InstanceType": "ml.m4.xlarge", 
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
        }
    ],
    DataCaptureConfig=
    {
            "EnableCapture": True,
            "InitialSamplingPercentage":100,
            "DestinationS3Uri":f"s3://{bucket_name}/{prefix}/datacapture/",
            "CaptureOptions":[{'CaptureMode': 'Input'}, {'CaptureMode': 'Output'}]
    },        
)

ClientError: An error occurred (ValidationException) when calling the CreateEndpointConfig operation: Cannot create already existing endpoint configuration "arn:aws:sagemaker:us-east-1:278578987671:endpoint-config/shadow-xgboost-epconfig".

<a id='deploy'></a>
## Deploy!

In [84]:
# Configuration object passed in when deploying Models to SM endpoints
data_capture_config = DataCaptureConfig(
    enable_capture = True, 
    sampling_percentage = 100, # Optional
    destination_s3_uri = f"s3://{bucket_name}/{prefix}/datacapture/", # Optional
    capture_options = ['REQUEST','RESPONSE']
)

In [133]:
predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge', data_capture_config=data_capture_config, serializer=sagemaker.serializers.CSVSerializer())

--------!

In [118]:
predictor2 = estimator2.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

--------!

In [134]:
predictor.endpoint_name

'sagemaker-xgboost-2023-01-26-18-19-35-745'

In [135]:
predictor2.endpoint_name

'rf-scikit-2023-01-26-17-42-26-068'

## Predict

Now when we send a prediction to the deployed endpoint, we will recieve a response from the production variant.  The shadow variant will also get the input payload.  

In [122]:
# single prediction
# We expect 4 - DDoS attacks-LOIC-HTTP as the predicted class.
test_values = [80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0]
test_values = np.array(test_values).reshape(1, -1)
result = predictor.predict(test_values)
print(result)

b'4.0'


In [120]:
# single prediction
# We expect 4 - DDoS attacks-LOIC-HTTP as the predicted class.
test_values = [80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0]
test_values = np.array(test_values).reshape(1, -1)
result = predictor2.predict(test_values)
print(result)

[4]


In [116]:
np.array(test_values).reshape(1, -1).shape

(1, 84)

In [49]:
preds = []
for index, row in holdout.iterrows():
    arg = ','.join(map(str, row.to_numpy()[1::]))
    pred = sm_runtime.invoke_endpoint(EndpointName=predictor.endpoint_name, 
                                      ContentType='text/csv',
                                      Body=arg)['Body'].read()
    true_val = row.Target
    preds.append([pred, true_val])

In [40]:
sm_runtime = boto3.Session().client("sagemaker-runtime")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [ ]:
# We expect 4 - DDoS attacks-LOIC-HTTP as the predicted class for this instance.
test_values = "80,1056736,3,4,20,964,20,0,6.666666667,11.54700538,964,0,241.0,482.0,931.1691850999999,6.6241710320000005,176122.6667,431204.4454,1056315,2,394,197.0,275.77164469999997,392,2,1056733,352244.3333,609743.1115,1056315,24,0,0,0,0,72,92,2.8389304419999997,3.78524059,0,964,123.0,339.8873763,115523.4286,0,0,1,1,0,0,0,1,1.0,140.5714286,6.666666667,241.0,0.0,0.0,0.0,0.0,0.0,0.0,3,20,4,964,8192,211,1,20,0.0,0.0,0,0,0.0,0.0,0,0,20,2,2018,1,0,1,0"

In [42]:
response = sm_runtime.invoke_endpoint(EndpointName=predictor.endpoint_name, 
                                      ContentType='text/csv',
                                      Body=arg)

In [43]:
response['Body'].read()

b'11.0'

In [46]:
holdout.iloc[10].Target

11.0

<a id='shadow'></a>
# Create a Shadow Test 

## Create a Shadow Test using an Existing Endpoint

Now we will create a shadow test using the existing endpoint.  We will stop this test using the API.  Note that we could also specify the test start and stop time when we create the inference experiements.  If we don't provide the start and end times then the experiment starts immediately and concludes after 7 days.  We are using an existing endpoint for this test.  SageMaker will update that endpoint with the new variants.  The production endpoint will also update the inference compute for the production variant


In [91]:
predictor.

EndpointContext(sagemaker_session=<sagemaker.session.Session object at 0x7f701c53e490>,context_name='sagemaker-xgboost-2023-01-25-22-58-32-924-1674687513-aws-endpoint',context_arn='arn:aws:sagemaker:us-east-1:431615879134:context/sagemaker-xgboost-2023-01-25-22-58-32-924-1674687513-aws-endpoint',source=ContextSource(source_uri='arn:aws:sagemaker:us-east-1:431615879134:endpoint/sagemaker-xgboost-2023-01-25-22-58-32-924',source_type='ARN',source_id='Wed Jan 25 22:58:33 UTC 2023'),context_type='Endpoint',properties={'DomainArn': 'arn:aws:sagemaker:us-east-1:431615879134:domain/d-179iphws0rc9', 'Status': 'InService', 'UserProfileArn': 'arn:aws:sagemaker:us-east-1:431615879134:user-profile/d-179iphws0rc9/jupyterlab3'},creation_time=datetime.datetime(2023, 1, 25, 22, 58, 33, tzinfo=tzlocal()),created_by={},last_modified_time=datetime.datetime(2023, 1, 25, 23, 2, 20, 258000, tzinfo=tzlocal()),last_modified_by={},lineage_group_arn='arn:aws:sagemaker:us-east-1:431615879134:lineage-group/sagemak

In [159]:
shadowtestname = 'ShadowInferenceTestExistingEP'
infexperimentarn = sm_client.create_inference_experiment(
    Name=shadowtestname,
    Type='ShadowMode',
    Description='Shadow inference test created via boto3 python API using an existing EP',
    RoleArn=role,
    EndpointName=predictor.endpoint_name,
    ModelVariants=[
        {
            'ModelName': model_name1,
            'VariantName': 'AllTraffic',
            'InfrastructureConfig': {
                'InfrastructureType':'RealTimeInference',
                'RealTimeInferenceConfig': {
                    'InstanceType': 'ml.m4.xlarge',
                    'InstanceCount': 1 
                }
            }
        },
        
        {
            'ModelName': model_name2,
            'VariantName': 'Shadow-01',
            'InfrastructureConfig': {
                'InfrastructureType':'RealTimeInference',
                'RealTimeInferenceConfig': {
                    'InstanceType': 'ml.m4.xlarge',
                    'InstanceCount': 1 
                }
            }
        },
    ],
    DataStorageConfig={
        'Destination':f's3://{bucket_name}/{prefix}/datacapture_test/',
    },
    ShadowModeConfig={
        'SourceModelVariantName': 'AllTraffic',
        'ShadowModelVariants': [
            {
                'ShadowModelVariantName': 'Shadow-01',
                'SamplingPercentage': 100
            },
        ]
    },
)   


In [102]:
infexperimentarn

{'InferenceExperimentArn': 'arn:aws:sagemaker:us-east-1:431615879134:inference-experiment/shadowinferencetestexistingep',
 'ResponseMetadata': {'RequestId': 'a9f2ae57-1857-49cd-930d-585260586030',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a9f2ae57-1857-49cd-930d-585260586030',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '120',
   'date': 'Thu, 26 Jan 2023 14:39:42 GMT'},
  'RetryAttempts': 0}}

In [104]:
sm_client.describe_inference_experiment(Name=shadowtestname)

{'Arn': 'arn:aws:sagemaker:us-east-1:431615879134:inference-experiment/shadowinferencetestexistingep',
 'Name': 'ShadowInferenceTestExistingEP',
 'Type': 'ShadowMode',
 'Schedule': {'StartTime': datetime.datetime(2023, 1, 26, 14, 39, 42, 101000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 2, 2, 14, 39, 42, 101000, tzinfo=tzlocal())},
 'Status': 'Creating',
 'Description': 'Shadow inference test created via boto3 python API using an existing EP',
 'CreationTime': datetime.datetime(2023, 1, 26, 14, 39, 41, 503000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 1, 26, 14, 39, 42, 80000, tzinfo=tzlocal()),
 'RoleArn': 'arn:aws:iam::431615879134:role/sagemaker-test-role',
 'EndpointMetadata': {'EndpointName': 'sagemaker-xgboost-2023-01-25-22-58-32-924',
  'EndpointConfigName': 'sagemaker-xgboost-2023-01-25-22-58-32-924',
  'EndpointStatus': 'Updating'},
 'ModelVariants': [{'ModelName': 'PROD-XGBoost-Webtraffic',
   'VariantName': 'AllTraffic',
   'Infrastructure

In [138]:
sm_client.stop_inference_experiment(
    Name=shadowtestname,
    ModelVariantActions={
        'string': 'Remove'
    },
    DesiredState='Cancelled',
    Reason='string'
)

ClientError: An error occurred (ValidationException) when calling the StopInferenceExperiment operation: Cannot stop InferenceExperiment in Creating status

## New Shadow Experiment with a New Endpoint

In [ ]:
datetime.now()

In [ ]:
infexperimentarn = sm_client.create_inference_experiment(
    Name='ShadowInferenceTestNEWEP',
    Type='ShadowMode',
    Schedule={
        'StartTime': datetime(2023, 1, 10, 20, 27, 0),
        'EndTime': datetime(2023, 1, 16, 22, 46, 10)
    },
    Description='Shadow interence test created via boto3 python API',
    RoleArn=role,
    EndpointName='shadowTestEPCodeNewEP',
    ModelVariants=[
        {
            'ModelName': 'PROD-XGBoost-Webtraffic',
            'VariantName': 'Production-01',
            'InfrastructureConfig': {
                'InfrastructureType': 'RealTimeInference',
                'RealTimeInferenceConfig': {
                    'InstanceType': 'ml.m5.xlarge',
                    'InstanceCount': 1
                }
            },
        },
        {
            'ModelName': 'SHADOW-XGBoost-Webtraffic',
            'VariantName': 'Shadow-01',
            'InfrastructureConfig': {
                'InfrastructureType': 'RealTimeInference',
                'RealTimeInferenceConfig': {
                    'InstanceType': 'ml.m4.xlarge',
                    'InstanceCount': 1
                }
            },
        }
    ],
    DataStorageConfig={
        'Destination':f's3://{bucket_name}/{prefix}/datacapture_test/',
    },
    ShadowModeConfig={
        'SourceModelVariantName': 'Production-01',
        'ShadowModelVariants': [
            {
                'ShadowModelVariantName': 'Shadow-01',
                'SamplingPercentage': 80
            }
        ]
    },
)   

In [ ]:
sm_client.describe_inference_experiment(Name='ShadowTestConsoleNewEP')

In [ ]:
sm_client.describe_inference_experiment(Name='ShadowTestExistingConsole')

<a id='eval'></a>
# Evaluate

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request. But first, we'll need to setup serializers and deserializers for passing our test_data NumPy arrays to the model behind the endpoint.


In [ ]:
model1_predicted_classes = []
for index, row in holdout.iterrows():
    val = serialize(row)
    prediction = predictor.predict(val)
    _class = deserialize(prediction)
    model1_predicted_classes.append(_class)

Since our Shadow test was running when we sent the data to our endpoint, we can get the test model's predictions from S3

In [ ]:
import s3fs
shadow_result_location = "S3://path/to/shadow/data/results.csv"
model2_predicted_classes 


Define the metric calculation functions

In [98]:
def true_positive(y_true, y_pred):
    tp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1
    return tp


def false_positive(y_true, y_pred):
    fp = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 0 and yp == 1:
            fp += 1
    return fp


def false_negative(y_true, y_pred):
    fn = 0
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 0:
            fn += 1
    return fn


# y_true: an iterable of actual values from a multi-class validation set
# y_pred: an iterable of predicted values from the same set
def evaluate(y_true, y_pred):
    num_classes = len(y_true.unique())
    precision = 0
    recall = 0
    f1 = 0

    for _class in list(holdout.Target.unique()):
        temp_true = [1 if p == int(_class) else 0 for p in y_true]
        temp_pred = [1 if p == int(_class) else 0 for p in y_pred]

        true_pos = true_positive(temp_true, temp_pred)
        false_pos = false_positive(temp_true, temp_pred)
        false_neg = false_negative(temp_true, temp_pred)

        # find the precision, recall, and F1 for each class
        temp_precision = true_pos / (true_pos + false_pos + 1e-6)
        precision += temp_precision
        temp_recall = true_pos / (true_pos + false_neg + 1e-6)
        recall += temp_recall
        temp_f1 = 2 * temp_precision * temp_recall / (temp_precision + temp_recall + 1e-6)
        f1 += temp_f1

    # Find the mean of each evaluation metric over all the classes
    precision /= num_classes
    recall /= num_classes
    f1 /= num_classes

    return {'precision': precision, 'recall': recall, 'f1': f1}

In [ ]:
# This is the true results for both models
true_class_values = holdout.Target.values
# This will be the production predictions
model1_predicted_classes = []
# This will be our Shadow test predictions
model2_predicted_classes = [] # load from S3

production_metrics = evaluate(true_class_values, model1_predicted_values)
shadow_metrics = evaluate(true_class_values, model2_predicted_values)

print(production_metrics['f1'])
print(shadow_metrics['f1'])

Our Shadow metrics look pretty good, let's promote that model to production:

In [ ]:
# Promote 

If we discover problems after promoting our test model, we can easily roll back:

In [ ]:
# roll back to n-1

## Simulate Production Traffic

We will now simulate the production traffic.  We will loop over the production data.  In a real production use case you won't need to do this since actual production data will be flowing to the production endpoint.  

Note that we are not capturing the inference request or output.  We could do this, however, we have configured data capture so instead we let SageMaker handle the data capture to s3.

In [ ]:
for p in prod:
    predictor.predict(p).decode('utf-8')



Now, we'll use a simple function to:

1. Loop over our test dataset
2. Split it into mini-batches of rows
3. Convert those mini-batchs to CSV string payloads
4. Retrieve mini-batch predictions by invoking the XGBoost endpoint
5. Collect predictions and convert from the CSV output our model provides into a NumPy array



In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [ ]:
predictions = predict(val_df.to_numpy()[:,1:])

In [ ]:
predictions.shape

In [ ]:
actual = val_df.to_numpy()[:,0]

In [ ]:
actual.shape

In [ ]:
class_list = ['Benign','Bot','DoS attacks-GoldenEye','DoS attacks-Slowloris','DDoS attacks-LOIC-HTTP','Infilteration','DDOS attack-LOIC-UDP','DDOS attack-HOIC','Brute Force-Web','Brute Force-XSS','SQL Injection','DoS attacks-SlowHTTPTest','DoS attacks-Hulk','FTP-BruteForce','SSH-Bruteforce']
fig, ax = plt.subplots(figsize=(15,10))
cm = confusion_matrix(actual,predictions)
normalized_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(normalized_cm, ax=ax, annot=cm, fmt='',xticklabels=class_list,yticklabels=class_list)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confustion Matrix')
plt.show()

Finally, let's gracefully stop the deployed endpoint.

TODO - PROMOTE THE SHADOW VARIANT TO PROD VIA CODE, INCLUDE CODE ON HOW TO ROLL BACK TO N-1 VERSION OF THE MODEL

In [132]:
#predictor.delete_endpoint()
sm_client.delete_inference_experiment(
    Name=shadowtestname
)

ClientError: An error occurred (ValidationException) when calling the DeleteInferenceExperiment operation: Cannot delete InferenceExperiment with status Creating.

# References

* A Realistic Cyber Defense Dataset (CSE-CIC-IDS2018) - https://registry.opendata.aws/cse-cic-ids2018/
* AIM362 - Re:Invent 2019 SageMaker Debugger and Model Monitor - https://github.com/aws-samples/reinvent2019-aim362-sagemaker-debugger-model-monitor